# Backcasting Demo Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pickle
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from delphi.visualization import visualize
import delphi.jupyter_tools as jt
import numpy as np
import pandas as pd
from scipy import stats
#Comment out the next line if you do not have the delphi.db file. 
from delphi.db import engine
jt.print_commit_hash_message()
import random as rm
import delphi.evaluation as EN
import warnings
#warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from indra.statements import (
    Concept,
    Influence,
    Evidence,
    Event,
    QualitativeDelta,
)
from delphi.utils.indra import *
from delphi.utils.shell import cd
import seaborn as sns
import matplotlib.pyplot as plt
from delphi.utils.fp import flatMap, take, ltake, lmap, pairwise, iterate, exists
from delphi.cpp.AnalysisGraph import AnalysisGraph as AG

Here I will set random seeds

In [ ]:
np.random.seed(87)
rm.seed(87)

In [ ]:
statements = [ (("large", -1, "UN/entities/human/financial/economic/inflation"),("small", 1, "UN/events/human/human_migration"))]


Now we load the Causal Analysis Graph (CAG). This is CAG was inferred by reading in a JSON corpus and was pruned and adjusted to be human migration centered. Also is a list of the nodes contained in the CAG

In [ ]:
#with open("../scripts/build/migration_centered_CAG.pkl",'rb') as f:
    #G = pickle.load(f)

G = AG.from_statements(statements)

In [ ]:
G.add_node()

In [ ]:
G.print_nodes()

In [ ]:
n = 2
query = " ".join(
        [
            "select Source, Indicator from concept_to_indicator_mapping",
            "where `Concept` like 'UN/entities/human/financial/economic/inflation'",
        ]
    )

results = engine.execute(query)

pd.DataFrame(results, columns=results.keys())

Next we map indicator variables to nodes. For the most part indicator variables can be inferred from available data and texts, but we can also manually map indicators to nodes. There is also a list of the indicator variables in the same order as the list of nodes above (i.e "Claims on other sectors of the domestic economy" is attached to "UN/events/human/economic_crisis".

In [ ]:
query = " ".join(
        [
            f"select * from indicator",
            f"where `Variable` like 'New asylum seeking applicants'",
        ]
    )

results = engine.execute(query)

pd.DataFrame(results, columns=results.keys())